In [ ]:
import redcap
import pandas as pd
from temba_client.v2 import TembaClient
from temba_client.exceptions import TembaRateExceededError, TembaTokenError, TembaHttpError, TembaNoSuchObjectError
import math
import numpy as np
import json
from support_functions import * 

Define classes to contain the different methods that will be used

In [ ]:
redcap = RedCap()
rapidpro = rapidPro()

In [ ]:
redcap.read_redcap_credentials_file(credential_file = 'redcapcredentials.json',form = 'asos2_investigator_phase1')
rapidpro.read_rapidpro_credentials_file(credential_file = 'rapidprocredentials.json',rapidpro_flow_id = 'bcaf334d-0fe0-4b45-8adb-1647aa3318c5')

Get the record data and metadata

In [ ]:
data_records = redcap.get_records()
data_metadata = redcap.get_metadata()
difference = redcap.get_difference_fields()


Next we need to parse the phone number from the Redcap database and check it works -- it needs to be converted into the correct format for the message sender

In [ ]:
import phonenumbers
import types

for index,row in data_records.iterrows():
    wa_number = row['whatsapp_number']
    f_wa_number = phonenumbers.parse(wa_number, None)
    phone_number = ('tel:'+phonenumbers.format_number(f_wa_number,
                                                      phonenumbers.PhoneNumberFormat.E164))
    print(phone_number)
    contacts = rapidpro.client.get_contacts(urn = phone_number)
    #if the contact exists then make sure it is in the correct group
    if contacts.first() is None:
        print('contact doesnt exist- creating')
        rapidpro.client.create_contact(urns = [phone_number])
        rapidpro.add_contact_to_groups(phone_number,'asos-investigators')
    else:
        print('contact already exists- just updating')
        rapidpro.add_contact_to_groups(phone_number,'asos-investigators')
    number_missing_fields = len(difference[index])
    print(number_missing_fields)
    name = row['name']
    if(number_missing_fields > 0):
        string = 'Thanks {name} for helping us by registering for the ASOS-2 trial. We have noticed that there are {missing_vals} missing fields in your latest survey. Would you mind completing the registration when you are able? Thanks!'.format(missing_vals=number_missing_fields,name=name)
    else:
        string = 'Thanks {name} for helping us by registering for the ASOS-2 trial. You have completed the latest survey. Thanks!'.format(form_name = redcap.form,name=name)

    print(string)
    message_dict = {'wa_message' : string }
    rapidpro.add_fields_to_contact(phone_number,message_dict)
    rapidpro.start_flow(phone_number)

Here we update all the contacts as necessary